# Geração de texto sobre Ordem Paranormal
Esse projeto teve como objetivo exercitar minha prática com o uso do tensorflow e machine learning no geral.
Esse jupyter notebook serve como forma de treinar o modelo ou como rodar o modelo já treinado usando a base de dados.
Vamos começar importanto tudo que é necessário para rodar o script.
Esse modelo tenta prever palavras (tokens).

In [1]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow.keras.utils as ku
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tensorflow.keras.models import load_model
from gensim.models import KeyedVectors
import os

In [2]:
#Essas flags só retiram os warning tanto do Numpy quanto o TensorFlow.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  
np.seterr(divide = 'ignore') 

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
data = pd.read_csv('banco de dados/s1_and_s2_mexido_comacento.csv', sep="#")
print(data.columns)

def preprocess(text):
	text_input = re.sub('[^a-zA-Z1-9à-úÀ-Ú]+', ' ', str(text))
	output = re.sub(r'\d+', '',text_input)

	return output.lower().strip()

data['text'] = data.text.map(preprocess)
corpus_cleaned = data['text'].astype(str).values.tolist()

Index(['text'], dtype='object')


In [5]:

sentences = []
sentence_length = []

for item in corpus_cleaned:
	word_list = item.split()
	sentences.append(item)
	number_of_words = len(word_list)
	sentence_length.append(number_of_words)

sentences = pd.DataFrame(sentences)
sentence_length = pd.DataFrame(sentence_length)

sentences = sentences.merge(sentence_length, left_index=True, right_index=True)

sentences['number_of_words']=sentences['0_y']
sentences['sentence']=sentences['0_x']
sentences=sentences[['sentence','number_of_words']]
sentences['number_of_words']=sentences['number_of_words'].astype(int)

#Usando sentenças de 6 palavras porque são as que mais aparecem no banco de dados
sentences = sentences[sentences['number_of_words'] == 6]
corpus_cleaned=sentences['sentence'].values.tolist()
corpus = corpus_cleaned

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

input_sequences = []

for review in corpus:
	token_list = tokenizer.texts_to_sequences([review])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
total_words = len(tokenizer.word_index) + 1
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)
print(total_words)

5314


Essa parte serve para treinar o modelo.

In [13]:
model = Sequential()
model.add(Embedding(total_words, 240, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, activation='tanh', return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=10), ModelCheckpoint(filepath='modelos/model_s1s2_mexido_comacento.h5', monitor='accuracy', save_best_only=True)]

Essa parte serve para rodar o modelo.
Caso esteja rodando direto do modelo salvo (extensão .h5), rode a parte de baixo, senão continue.

In [14]:
history = model.fit(predictors, label, epochs=300, verbose=1, callbacks=[callbacks])

Epoch 1/300
 134/1510 [=>............................] - ETA: 18s - loss: 7.3094 - accuracy: 0.0259

KeyboardInterrupt: 

In [15]:
model = load_model('modelos/model_s1s2_mexido_comacento.h5')

In [16]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

continua = True
while continua:
	seed_text_one = input("Digite o começo da frase (o máximo será 6 palavras contando com as que você digitar):")


	seed_text = seed_text_one

	if seed_text == "":
		break
	
	tam = len(seed_text.split(" "))

    #Ter somente 6 palavras geram resultados melhores, mais de 6 começa a ficar menos coeso.
	next_words = 6-tam

	for temp in [0.2, 0.5, 1.0, 1.2]:
		text = seed_text
		print('Gerando com temperatura: ', temp)
		for _ in range(next_words):
			token_list = tokenizer.texts_to_sequences([text])[0]
			token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
			predicted = model.predict(token_list, verbose=0)[0]
			predicted = sample(predicted, temperature=temp)
			output_word = ""
			for word, index in tokenizer.word_index.items():
				if index == predicted:
					output_word = word
					break
			text += " " + output_word

		text = (text).capitalize()
		print(text)

Gerando com temperatura:  0.2
Cellbit: vocês já podem ser cocô
Gerando com temperatura:  0.5
Cellbit: você tá vendo o bosque
Gerando com temperatura:  1.0
Cellbit: boa noite senhoras e senhores
Gerando com temperatura:  1.2
Cellbit: boa noite senhoras e senhores
Gerando com temperatura:  0.2
Rakin: eu vou baixando a espingarda
Gerando com temperatura:  0.5
Rakin: eu quero lavar meu sobretudo
Gerando com temperatura:  1.0
Rakin: oi oi oi tô nervoso
Gerando com temperatura:  1.2
Rakin: em voltas de dentro dele
